In [1]:
# Nustatymai, parametrai:

item_type = 'mobilieji-telefonai'
# 'dronai', 'televizoriai', 'saldytuvai', 'televizoriai'
item_params = 'f.s-gamintojas=Samsung&sort=price&'

table_version = '_5'
item_producer = 'samsung'

url = f'https://www.varle.lt/{item_type}/?{item_params}'
print(url)

https://www.varle.lt/mobilieji-telefonai/?f.s-gamintojas=Samsung&sort=price&


In [ ]:
# Šis fragmentas automatiškai išvalo 'items_df' sukauptus duomenis,
# Pasirengta naujų duomenų įrašymui.

items_df = pd.DataFrame(columns=('id','price','url','price_in_cents','producer'))
# čia bus šios programos rezultatas

In [ ]:
# Pradėta 2024-11-21_ObjectProperties.ipynb
# Tęsinys. Čia 2024-11-28_varle_lt.ipynb

import pandas as pd
import time
import sqlite3
import random

from bs4 import BeautifulSoup

# Selenium
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium import webdriver

from my_lib.properties_of_4 import properties_of
from my_lib.my_db_functions import save_df_to_sqlite

In [ ]:
# Atidaro Chrome naršyklę,
# reikia patvirtinti "Leisti visus slapukus"!
# Šis fragmentas automatiškai panaikina „draiver“.

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)

# url = f'https://www.varle.lt/{item_type}/?{item_params}' # iškeliamas į pradžią?

driver.get(url)
# time.sleep(3)
source = driver.page_source
print(f'First page: {url}')

bs = BeautifulSoup(source,'html.parser')

# Pradedama html analizė. Nuskaitomas puslapių sk.
pages_numbers = bs.find('div',{'class':'PAGINATION'}).find_all('li',{'class':'number'})
total_pages = int(pages_numbers[-1].text.strip())
print(f'Total pages: {total_pages}')

In [ ]:
# Toliau analizuojamas prekių sąrašo html iš bs=BeautifulSoup().

items_blocks = bs.find_all('div',{'class':'GRID_ITEM'})
for item_block in items_blocks:

    item_href = item_block.find('div',{'class':'product-title'}).find('a')['href']
    item_url = 'https://www.varle.lt'+ item_href
    item_id = item_href.split('-')[-1].split('.')[0]

    price_container = item_block.find('div',{'class':'price-container'})
    price_value = price_container.find('span',{'class':'price-value'}).getText()
    item_price = ''.join(price_value.split())

    items_df.loc[len(items_df)] = {'id':int(item_id),'price':item_price,'url':item_url,'producer':item_producer}

print(f'DF items: {len(items_df)}.')

DF items: 38.


In [ ]:
items_df

In [9]:
f'{item_type}{table_version}'

'mobilieji-telefonai_5'

In [ ]:
# Perskaitomi kiti prekių sąrašo puslapiai,
# po to įrašoma į db

import random

# for page_number in range(2,total_pages+1):
for page_number in range(2,total_pages+1):

    url = f'https://www.varle.lt/{item_type}/?{item_params}p={page_number}'
    driver.get(url)
    time.sleep(random.randint(25,75))
    source = driver.page_source
    print(f'Prskaityta {url}')

    bs = BeautifulSoup(source,'html.parser')

    items_blocks = bs.find_all('div',{'class':'GRID_ITEM'})
    for item_block in items_blocks:
        item_href = item_block.find('div',{'class':'product-title'}).find('a')['href']
        item_url = 'https://www.varle.lt'+item_href
        item_id = item_href.split('-')[-1].split('.')[0]

        price_container = item_block.find('div',{'class':'price-container'})
        price_value = price_container.find('span',{'class':'price-value'}).getText()
        item_price = ''.join(price_value.split())

        # items_df.loc[len(items_df)] = {'id':int(item_id),'price':item_price,'url':item_url}
        items_df.loc[len(items_df)] = {'id':int(item_id),'price':item_price,'url':item_url,'producer':item_producer}

    items_total = len(items_df)
    items_df.drop_duplicates(subset='id',keep='last',inplace=True)
    print(f'Get {items_total} items. Unique {len(items_df)} items.')

# driver.close() # Uždaro Chrome puslapį.

Prskaityta https://www.varle.lt/mobilieji-telefonai/?f.s-gamintojas=Samsung&sort=price&p=2
Get 76 items. Unique 76 items.
Prskaityta https://www.varle.lt/mobilieji-telefonai/?f.s-gamintojas=Samsung&sort=price&p=3
Get 114 items. Unique 114 items.
Prskaityta https://www.varle.lt/mobilieji-telefonai/?f.s-gamintojas=Samsung&sort=price&p=4
Get 152 items. Unique 152 items.
Prskaityta https://www.varle.lt/mobilieji-telefonai/?f.s-gamintojas=Samsung&sort=price&p=5
Get 190 items. Unique 190 items.
Prskaityta https://www.varle.lt/mobilieji-telefonai/?f.s-gamintojas=Samsung&sort=price&p=6
Get 228 items. Unique 228 items.
Prskaityta https://www.varle.lt/mobilieji-telefonai/?f.s-gamintojas=Samsung&sort=price&p=7
Get 266 items. Unique 266 items.
Prskaityta https://www.varle.lt/mobilieji-telefonai/?f.s-gamintojas=Samsung&sort=price&p=8
Get 280 items. Unique 280 items.
Lentelė 'mobilieji-telefonai_5' sėkmingai įrašyta į DB.


,id,price,url,price_in_cents,producer
0,34459954,54.99€,https://www.varle.lt/mobilieji-telefonai/samsu...,5499,samsung
1,34802720,54.99€,https://www.varle.lt/mobilieji-telefonai/samsu...,5499,samsung
2,30802672,92.11€,https://www.varle.lt/mobilieji-telefonai/ecost...,9211,samsung
3,32480257,103.81€,https://www.varle.lt/mobilieji-telefonai/samsu...,10381,samsung
4,31806515,115.99€,https://www.varle.lt/mobilieji-telefonai/samsu...,11599,samsung
...,...,...,...,...,...
275,22970524,1989.99€,https://www.varle.lt/mobilieji-telefonai/isman...,198999,samsung
276,22867446,2099.99€,https://www.varle.lt/mobilieji-telefonai/samsu...,209998,samsung
277,37051688,2139€,https://www.varle.lt/mobilieji-telefonai/samsu...,213900,samsung
278,37054437,2139€,https://www.varle.lt/mobilieji-telefonai/samsu...,213900,samsung


In [ ]:
# Pridedamas kainos stulpelis centais (int)
import re

def price_in_cents(price:str) -> int:
    # Pašalina visus simbolius, išskyrus skaičius, kablelį ir tašką:
    cleaned = re.sub(r'[^\d.,]', '', price).replace(',', '.').replace(' ', '')
    return int(float(cleaned)*100)

items_df['price_in_cents'] = items_df['price'].apply(price_in_cents)

# Save to SQLite3
save_df_to_sqlite(items_df,"./my_scrap_working_data.db",table_name=f'{item_type}{table_version}')

items_df

In [9]:
import pandas as pd
import sqlite3
import inspect
from typing import Union

def df_to_sqlite(df: pd.DataFrame, db: Union[str, sqlite3.Connection], table_name: str = None) -> None:
    """
    Įrašo Pandas DataFrame į nurodytą SQLite3 duomenų bazę.

    Args:
        df (pd.DataFrame): Duomenys, kurie bus įrašyti į DB.
        db (Union[str, sqlite3.Connection]): Kelias į SQLite DB failą arba jau egzistuojantis ryšys.
        table_name (str, optional): Lentelės pavadinimas. Jei nenurodytas, bandoma naudoti kintamojo pavadinimą.

    Raises:
        ValueError: Jei lentelės pavadinimo nepavyksta nustatyti automatiškai.
    """
    if not isinstance(df, pd.DataFrame):
        raise ValueError("Pateiktas objektas nėra DataFrame.")

    # Nustatome lentelės pavadinimą, jei jis nenurodytas
    if table_name is None:
        # Bandome atspėti kintamojo pavadinimą
        frame = inspect.currentframe().f_back
        table_name = next(
            (name for name, val in frame.f_globals.items() if val is df),
            None
        )
        if table_name is None:
            raise ValueError("Nepavyko nustatyti lentelės pavadinimo. Prašome jį nurodyti rankiniu būdu.")

    # Atidarome DB ryšį, jei reikia
    connection_provided = isinstance(db, sqlite3.Connection)
    connection = db if connection_provided else sqlite3.connect(db)

    try:
        # Įrašome duomenis į DB
        df.to_sql(table_name, connection, if_exists='replace', index=False)
        print(f"Lentelė '{table_name}' sėkmingai įrašyta į DB.")
    except Exception as e:
        print(f"Klaida įrašant lentelę '{table_name}': {e}")
        raise
    finally:
        # Uždaryti ryšį, jei jį atidarėme funkcijoje
        if not connection_provided:
            connection.close()

def sqlite_to_df(db: Union[str, sqlite3.Connection], table_name: str) -> pd.DataFrame:
    """
    Nuskaito lentelę iš SQLite DB ir grąžina ją kaip Pandas DataFrame.

    Args:
        db (Union[str, sqlite3.Connection]): Kelias į SQLite DB failą arba jau egzistuojantis ryšys.
        table_name (str): Lentelės pavadinimas, kurią reikia nuskaityti.

    Returns:
        pd.DataFrame: Nuskaityti duomenys kaip DataFrame.

    Raises:
        ValueError: Jei lentelės pavadinimas nepateiktas.
        Exception: Jei nuskaitymas nepavyko.
    """
    if not table_name:
        raise ValueError("Reikia nurodyti lentelės pavadinimą.")

    # Atidarome DB ryšį, jei reikia
    connection_provided = isinstance(db, sqlite3.Connection)
    connection = db if connection_provided else sqlite3.connect(db)

    try:
        # Skaitome lentelę į DataFrame
        df = pd.read_sql_query(f"SELECT * FROM `{table_name}`", connection)
        print(f"Lentelė `{table_name}` sėkmingai nuskaityta iš DB.")
        return df
    except Exception as e:
        print(f"Klaida skaitant lentelę `{table_name}`: {e}")
        raise
    finally:
        # Uždaryti ryšį, jei jį atidarėme funkcijoje
        if not connection_provided:
            connection.close()


In [ ]:
# Save to SQLite3
df_to_sqlite(items_df,"./my_scrap_working_data.db",table_name=f'{item_type}{teble_version}')

In [31]:
# restore DF from DB
items_df = sqlite_to_df("./my_scrap_working_data.db",item_type)

Lentelė `mobilieji-telefonai` sėkmingai nuskaityta iš DB.


In [37]:
items_df

,id,price,url,price_in_cents
0,31840142,259€,https://www.varle.lt/mobilieji-telefonai/isman...,25900
1,31784540,998.99€,https://www.varle.lt/mobilieji-telefonai/isman...,99899
2,31565733,358.99€,https://www.varle.lt/mobilieji-telefonai/isman...,35899
3,16752381,549€,https://www.varle.lt/mobilieji-telefonai/apple...,54900
4,35996699,148.99€,https://www.varle.lt/mobilieji-telefonai/isman...,14899
...,...,...,...,...
1893,40796802,44.99€,https://www.varle.lt/mobilieji-telefonai/iget-...,4499
1894,30111701,169.99€,https://www.varle.lt/mobilieji-telefonai/iget-...,16999
1895,14190359,99.24€,https://www.varle.lt/mobilieji-telefonai/maxco...,9924
1898,38450528,774.18€,https://www.varle.lt/mobilieji-telefonai/mobil...,77418
